In [1]:
spark.stop()

In [3]:
from pyspark.sql import SparkSession

spark=SparkSession.builder\
.appName('Data Integration')\
.master('yarn')\
.getOrCreate()

25/09/28 22:47:11 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.Con

In [4]:
spark

In [ ]:
data_path='/data/olist/'
customer_df=spark.read.csv(data_path+'olist_customers_dataset.csv',header=True,inferSchema=True)
orders_df=spark.read.csv(data_path+'olist_orders_dataset.csv',header=True,inferSchema=True)
order_items_df=spark.read.csv(data_path+'olist_order_items_dataset.csv',header=True,inferSchema=True)
payments_df=spark.read.csv(data_path+'olist_order_payments_dataset.csv',header=True,inferSchema=True)
reviews_df=spark.read.csv(data_path+'olist_order_reviews_dataset.csv',header=True,inferSchema=True)
geolocation_df=spark.read.csv(data_path+'olist_geolocation_dataset.csv',header=True,inferSchema=True)
sellers_df=spark.read.csv(data_path+'olist_sellers_dataset.csv',header=True,inferSchema=True)
product_df=spark.read.csv(data_path+'olist_products_dataset.csv',header=True,inferSchema=True)
category_translation_df=spark.read.csv(data_path+'product_category_name_translation.csv',header=True,inferSchema=True)

25/09/28 22:49:39 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/09/28 22:49:43 ERROR Inbox: Ignoring error                       (0 + 0) / 1]
java.lang.NullPointerException: null
	at org.apache.spark.scheduler.cluster.CoarseGrainedSchedulerBackend$DriverEndpoint$$anonfun$receiveAndReply$1.applyOrElse(CoarseGrainedSchedulerBackend.scala:359) ~[spark-core_2.12-3.5.3.jar:3.5.3]
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:103) [spark-core_2.12-3.5.3.jar:3.5.3]
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213) [spark-core_2.12-3.5.3.jar:3.5.3]
	at org.apache.spark.rpc.netty.Inbox.process(Inbox.scala:100) [spark-core_2.12-3.5.3.jar:3.5.3]
	at org.apache.spark.rpc.netty.MessageLoop.org$apache$spark$rpc$netty$MessageLoop$$receiveLoop(MessageLoop.scala:75) [spark-core_2.12-3.5.3.jar:3.5.3]
	at org.apache.spark.rpc.netty.MessageLoop$$anon$1.run(Mes

In [10]:
#cache frequently used data for better performance

orders_df.cache()
customer_df.cache()
order_items_df.cache()


DataFrame[order_id: string, order_item_id: int, product_id: string, seller_id: string, shipping_limit_date: timestamp, price: double, freight_value: double]

In [17]:
order_items_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)



In [4]:
order_items_joined_df=orders_df.join(order_items_df, 'order_id', 'inner')

In [5]:
order_items_products_df=order_items_joined_df.join(product_df, 'product_id', 'inner')

In [6]:
#this df has orders, all the items in the order, their product details and seller details
order_items_product_sellers_df=order_items_products_df.join(sellers_df, 'seller_id', 'inner')

In [7]:
full_orders_df=order_items_product_sellers_df.join(customer_df,'customer_id','inner')

In [9]:
#Geolocation data- Here we  join full_orders and geolcation but here we will do left join because we want all the full_order data though gelocation is not available.

full_orders_df=full_orders_df.join(geolocation_df, full_orders_df.customer_zip_code_prefix == geolocation_df.geolocation_zip_code_prefix,'left')

In [10]:
#joining full_orders and reviews - doing left again we won't have reviews to all orders so we might loose so much data if we do inner join
full_orders_df=full_orders_df.join(reviews_df, 'order_id','left')

In [12]:
full_orders_df= full_orders_df.join(payments_df,'order_id','left')

In [13]:
full_orders_df.cache()

25/09/27 18:22:14 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


DataFrame[order_id: string, customer_id: string, seller_id: string, product_id: string, order_status: string, order_purchase_timestamp: timestamp, order_approved_at: timestamp, order_delivered_carrier_date: timestamp, order_delivered_customer_date: timestamp, order_estimated_delivery_date: timestamp, order_item_id: int, shipping_limit_date: timestamp, price: double, freight_value: double, product_category_name: string, product_name_lenght: int, product_description_lenght: int, product_photos_qty: int, product_weight_g: int, product_length_cm: int, product_height_cm: int, product_width_cm: int, seller_zip_code_prefix: int, seller_city: string, seller_state: string, customer_unique_id: string, customer_zip_code_prefix: int, customer_city: string, customer_state: string, geolocation_zip_code_prefix: int, geolocation_lat: double, geolocation_lng: double, geolocation_city: string, geolocation_state: string, review_id: string, review_score: string, review_comment_title: string, review_commen

In [14]:
full_orders_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (null

In [24]:
# Total Revenues per seller
from pyspark.sql.functions import *
seller_revenue_df=full_orders_df.groupBy('seller_id').agg(sum('price').alias('totalseller_revenue')).orderBy('totalseller_revenue', ascending=False)
seller_revenue_df.show(5)

+--------------------+--------------------+
|           seller_id| totalseller_revenue|
+--------------------+--------------------+
|4869f7a5dfa277a7d...|3.6138717319999985E7|
|53243585a1d6dc264...| 3.429159294999998E7|
|4a3ca9315b744ce9f...| 3.375957084000012E7|
|7c67e1448b00f6e96...|3.2282321789999776E7|
|fa1c13f2614d7b5c4...|3.0139386309999995E7|
+--------------------+--------------------+
only showing top 5 rows



In [ ]:
#Top customer by spending

top_customer_df=full_orders_df.groupBy('customer_id').agg(sum('payment_value').alias('top_customers')).orderBy('top_customers',ascending=False)

# Optimized Joins for Data Integration

In [7]:
from pyspark.sql.functions import *

In [ ]:
order_items_joined_df=orders_df.join(order_items_df, 'order_id', 'inner')

In [ ]:
order_items_products_df=order_items_joined_df.join(product_df, 'product_id', 'inner')

In [14]:
#this df has small data so we can broadcast it. Broadcast means Spark sends a copy of the small DataFrame to every executor to avoid shuffling
order_items_product_sellers_df=order_items_products_df.join(broadcast(sellers_df), 'seller_id', 'inner')

In [15]:
full_orders_df=order_items_product_sellers_df.join(customer_df,'customer_id','inner')

In [16]:
#Geolocation data- Here we  join full_orders and geolcation but here we will do left join because we want all the full_order data though gelocation is not available.

full_orders_df=full_orders_df.join(broadcast(geolocation_df), full_orders_df.customer_zip_code_prefix == geolocation_df.geolocation_zip_code_prefix,'left')

In [17]:
#joining full_orders and reviews - doing left again we won't have reviews to all orders so we might loose so much data if we do inner join
full_orders_df=full_orders_df.join(broadcast(reviews_df), 'order_id','left')

In [18]:
full_orders_df= full_orders_df.join(payments_df,'order_id','left')

In [19]:
full_orders_df.cache()

25/09/28 21:04:50 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


DataFrame[order_id: string, customer_id: string, seller_id: string, product_id: string, order_status: string, order_purchase_timestamp: timestamp, order_approved_at: timestamp, order_delivered_carrier_date: timestamp, order_delivered_customer_date: timestamp, order_estimated_delivery_date: timestamp, order_item_id: int, shipping_limit_date: timestamp, price: double, freight_value: double, product_category_name: string, product_name_lenght: int, product_description_lenght: int, product_photos_qty: int, product_weight_g: int, product_length_cm: int, product_height_cm: int, product_width_cm: int, seller_zip_code_prefix: int, seller_city: string, seller_state: string, customer_unique_id: string, customer_zip_code_prefix: int, customer_city: string, customer_state: string, geolocation_zip_code_prefix: int, geolocation_lat: double, geolocation_lng: double, geolocation_city: string, geolocation_state: string, review_id: string, review_score: string, review_comment_title: string, review_commen

# Aggregations

In [44]:
#Total orders per customer- I kept counDistinct because for every item in the order(we joined orders, order_items) we have one row so we have duplicated order_id's in full_orders_df


customer_orders_df=full_orders_df.groupBy('customer_id').agg(countDistinct('order_id').alias('total_orders')).orderBy('total_orders', ascending=False)
customer_orders_df.show(truncate=False)

+--------------------------------+------------+
|customer_id                     |total_orders|
+--------------------------------+------------+
|23b672646bb77ec71ce0c7f3161bd131|1           |
|16cc3f64905e653b68d869cb6f63ba25|1           |
|ac106ae9d5ca35e4a7afa11fb83f11de|1           |
|afea86eaca7e4cfd9f4abb186e66c612|1           |
|b32b3f4b527cf062f07794f0aa0c5998|1           |
|b9ba9af563885055c7204c3b25357f0e|1           |
|ca85f1de72b224e6e79d9c959f6c384d|1           |
|2b64ee8770672fe8f26ee30b6830af94|1           |
|b48a2d34f7c271d312a654f068e344a4|1           |
|25f1a26f2cd003472ee7fc88e9da8adb|1           |
|136f8c475ac7abd11f3c8f93c24f88d8|1           |
|67260c60edf9d4c22831ad47718a0e5e|1           |
|b951f4aa14bd8cdbdaa9fe4bcc4f4750|1           |
|969769c94ebf3a6fa4c36060dbffab4e|1           |
|7b1d4d024295cc2b11b2896b06458baa|1           |
|6f7ed8a8514d8791f26e4b919cd28d4d|1           |
|e9d4dbce11413c598d7ba2ffb25c9030|1           |
|f22ebbb28922e5674081e7bff1d1baef|1     

In [52]:
"""Average reviews score per seller- this also has the same problem as previous like as we have duplicated order_id's so the sellerid's are
also duplicated so the review score calculation is wrong but i am doing normally
"""
#Avg review score per seller
order_seller_reviews = full_orders_df \
                   .dropDuplicates(["order_id","seller_id"])
seller_review_df=order_seller_reviews.groupBy('seller_id').agg(avg("review_score").alias('avg_reviewscore')).orderBy('avg_reviewscore', ascending=False)


seller_review_df.show(truncate=False)

+--------------------------------+---------------+
|seller_id                       |avg_reviewscore|
+--------------------------------+---------------+
|05730013efda596306417c3b09302475|5.0            |
|e5def42655b7490edac5a56fe8e9e603|5.0            |
|762fafdaa57a532ee0119731697579dd|5.0            |
|64c9a1db4e73e19aaafd3286dc448c96|5.0            |
|0b36063d5818f81ccb94b54adfaebbf5|5.0            |
|2c538755f1ca9540af144f266e70df6c|5.0            |
|0570350b23eda5444f8d1d9544932058|5.0            |
|4aba6a02a788d3ec81c03137144d9a80|5.0            |
|5e106d93b717c2682b9383da480364a1|5.0            |
|fd312b6bf05efac6c3772d5b52205d8a|5.0            |
|3e51f9cbe317bc3a88f2c76583811ee4|5.0            |
|2b2fed75b8e5ea3a052fc4c0825da485|5.0            |
|6548c2550b6e2d8f18053216e1c7b6f3|5.0            |
|19e0f40a907ec568c34bfac15de4f4a1|5.0            |
|e5545d4d4473876ea78c5bcebca365c3|5.0            |
|26b482dccfa29bd2e40703ba45523702|5.0            |
|6ebf4eceee4dd9847201c82e77ef81

In [61]:
# Most sold products (top 10)- there are chances in the same order particular item is bough multiple time so i kept count instead of countDistinct


popular_products=full_orders_df.groupBy('product_id').agg(count('order_id').alias('popular_product')).orderBy('popular_product', ascending=False).limit(10)
popular_products.show()

+--------------------+---------------+
|          product_id|popular_product|
+--------------------+---------------+
|aca2eb7d00ea1a7b8...|          86740|
|422879e10f4668299...|          81110|
|99a4788cb24856965...|          78775|
|389d119b48cf3043d...|          60248|
|d1c427060a0f73f6b...|          59274|
|368c6c730842d7801...|          58358|
|53759a2ecddad2bb8...|          52654|
|53b36df67ebb7c415...|          52105|
|154e7e31ebfa09220...|          42700|
|3dd2a17168ec895c7...|          40787|
+--------------------+---------------+



In [64]:
#Top customer by spending - if u keep payment_value we have to drop (customer_id, order_id) but if we use price we no need to drop payment_value=totalamount paint for order where price is each item in the order

top_customer_df=full_orders_df.groupBy('customer_id').agg((sum('price')+ sum("freight_value")).alias('top_customers')).orderBy('top_customers',ascending=False)
top_customer_df.show(10)

+--------------------+------------------+
|         customer_id|     top_customers|
+--------------------+------------------+
|d3e82ccec3cb5f956...| 7462522.800000073|
|df55c14d1476a9a34...|3678102.6199999982|
|fe5113a38e3575c04...| 3372700.919999999|
|63b964e79dee32a35...|2669979.8400000064|
|ec5b2ba62e5743423...|2597132.1600000006|
|46bb3c0b1a65c8399...|2466455.1999999983|
|05455dfa7cd02f13d...| 2213680.560000086|
|3690e975641f01bd0...|2159641.6399999997|
|349509b216bd5ec11...|1947618.4700000004|
|695476b5848d64ba0...| 1878003.809999994|
+--------------------+------------------+
only showing top 10 rows



# Windows function ad Ranking

In [67]:
from pyspark.sql.window import Window

In [72]:

# For each seller, pick their top 5 most expensive products 
window_spec=Window.partitionBy('seller_id').orderBy(desc('price'))


top_seller_products_df=full_orders_df.withColumn('rank', rank().over(window_spec)).filter(col('rank')<=5)
top_seller_products_df.show(5)


+--------------------+--------------------+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------+-------------------+-----+-------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+----------------------+-----------+------------+--------------------+------------------------+-------------------+--------------+---------------------------+-------------------+------------------+-------------------+-----------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+------------------+------------+--------------------+-------------+----+
|            order_id|         customer_id|           seller_id|          product_id|order_status|order_purchase_timestamp|  o

In [ ]:
#top 5 selling products per seller by revenue if it is by units  do count('order_items') instead of sum('price')
seller_product_revenue = (full_orders_df
    .groupBy("seller_id", "product_id")
    .agg(sum("price").alias("total_revenue"))
)


window_spec = Window.partitionBy("seller_id").orderBy(desc("total_revenue"))

top5_by_revenue = (seller_product_revenue
    .withColumn("rank", rank().over(window_spec))
    .filter(col("rank") <= 5)
)

top5_by_revenue.show(truncate=False)


# Advanced Aggregations and Enrichments

In [29]:
#Average order value(AOV) per customer

aov_df=full_orders_df.groupBy('customer_id').agg(
    countDistinct('order_id').alias('TotalOrder_perCustomer'),
    sum('price').alias('total_spend')).orderBy('TotalOrder_perCustomer', ascending=False)
aov_df=aov_df.withColumn('AOV_customer', round(col('total_spend')/col('TotalOrder_perCustomer'),2))

aov_df.show(truncate =False)


+--------------------------------+----------------------+------------------+------------+
|customer_id                     |TotalOrder_perCustomer|total_spend       |AOV_customer|
+--------------------------------+----------------------+------------------+------------+
|2b139a6842a26c3576fc0f5c940e1617|1                     |9394.0            |9394.0      |
|c0fe0fbc24994167dce810f83cb96890|1                     |16378.149999999927|16378.15    |
|7af0543c8b2d3e921f6337f59a72fb34|1                     |13580.0           |13580.0     |
|fab1a643ad6f3da348860df58bf39a24|1                     |2180.5            |2180.5      |
|89826e838207100cbb653011a5756080|1                     |1559.76           |1559.76     |
|9d1a96b181bc6f175146b254a2335e29|1                     |23099.670000000013|23099.67    |
|ad417eed258291b622014bf443a0633b|1                     |26649.0           |26649.0     |
|b5957b87d70d3b09ff0859bac2d2f12b|1                     |61662.0           |61662.0     |
|ed1ee9233

In [41]:
#Seller Performance metrics(Revenue, Average review score, ordercount(byunits))

order_seller_df=full_orders_df.dropDuplicates(['order_id', 'seller_id'])

seller_reviews =order_seller_df.groupBy('seller_id').agg(round(avg('review_score'),2).alias('avg_score'))

seller_revenue_orders=full_orders_df.groupBy('seller_id').agg(
    round(sum('price'),2).alias('Total_revenue'),
    count('order_id').alias('order_count')
)

seller_metrics_df = seller_revenue_orders.join(seller_reviews, on="seller_id", how="left")

seller_metrics_df.show(10, truncate=False)

+--------------------------------+-------------+-----------+---------+
|seller_id                       |Total_revenue|order_count|avg_score|
+--------------------------------+-------------+-----------+---------+
|8e6cc767478edae941d9bd9eb778d77a|1145757.4    |19024      |3.92     |
|4d600e08ecbe08258c79e536c5a42fee|436434.23    |724        |4.0      |
|62de60d81c55c29d76a6bce418b31cef|8343.0       |81         |4.0      |
|cb5ff1b9715e99589f3e0c7a6afab9e3|13235.0      |581        |4.0      |
|038b75b729c8a9a04b0d07cba7eabe86|17979.5      |77         |3.5      |
|b86a47b3366e3b5424a7b225781f4908|17646.0      |204        |3.0      |
|acadd4d36859671cb635527c6c977533|262034.0     |1275       |4.25     |
|33ab10be054370c254ddfcf0a6253422|45320.7      |849        |5.0      |
|b76dba6c951ab00dc4edf0a1aa88037e|302582.66    |19786      |4.18     |
|33cbbec1e7e1044aaf11d152172c776f|190508.32    |4659       |4.73     |
+--------------------------------+-------------+-----------+---------+
only s

In [42]:
full_orders_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (null

In [ ]:
#product popularity metrics - Units Sold, Revenue Generated, Number of Unique Orders,Number of Unique Customers


full_orders_df.groupBy('product_id','product_category_name').agg(
    count('order_item_id').alias('total_units_sold'),
    round(sum('price'),2).alias('total_revenue_perProduct'),
    countDistinct('order_id').alias('distinct_order'),
    countDistinct('customer_id').alias('distinct_customers'),
    countDistinct('seller_id').alias('unique_sellers')
).orderBy('total_units_sold', ascending=False).show(truncate=False)


In [ ]:
#Monthly revenue and order count trend - total_orders, total_revenue, avg_order_value, min_order_value,max_order_values

monthly_revenue = (full_orders_df
    .groupBy(month("order_purchase_timestamp").alias("month"))
    .agg(
        countDistinct("order_id").alias("total_orders"),
        sum("price").alias("total_revenue")
    )
    .withColumn("avg_order_value", round(col("total_revenue")/col("total_orders"), 2))
)


monthly_minmax = (full_orders_df
    .groupBy(F.month("order_purchase_timestamp").alias("month"))
    .agg(
        min("payment_value").alias("min_order_value"),
        max("payment_value").alias("max_order_value")
    )
)

# Join if you want one table
monthly_trend = monthly_revenue.join(monthly_minmax, on="month", how="inner")




In [ ]:
 
#customer Retention anlysis (first and last order)
full_orders_df.groupBy('customer_id').agg(
    min('order_purchase_timestamp').alias('first_order'),
    max('order_purchase_timestamp').alias('last_order')
)